In [1]:
import numpy as np
from astropy import units,constants
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d
from scipy.interpolate import interp1d
from scipy.integrate import simpson
import yaml
import os
import sys
import glob

sys.path.append('..')

from pyaxi_utils import load_single_result, get_kvals, get_times, n_p, n_k

output_dir='~/projects/pyAxiverse/data/piaxiverse_main1_SU3'  # Path to folder where subfolders/data files are saved

# example file: smooth resonance
file_smooth = 'piaxiverse_GMR_L4scan2_SU3_ac8fcc3fa093280e6916ba0d3e17f74a7ab2e8a2'
# example file: narrow resonance
file_narrow = 'piaxiverse_GMR_L4scan2_SU3_d9c1797fc0793cbd2a5bd8af36603fef3f1433a8'
# example file: steep resonance
file_steep  = 'piaxiverse_GMR_L4scan_SU3_211b7a1a8656bedb609b007e1cc8828d3991f3b8'
# example file: no resonance
file_none   = 'piaxiverse_GMR_L4scan_SU3_af2ee65ed6830a0aba14a1a3748f7ccfe47dcb70'

In [2]:
def resonance_check(t,nk,tmax=None):
    lnk = np.log10(nk)
    #print(np.all(np.isfinite(lnk)))
    if np.any(np.isinf(np.abs([nk[0],nk[1]]))):
        return 1e99,0.0,[1e99],[0.0]
    if np.all(lnk == -np.inf):
        return 0.0,t,np.zeros_like(t),t
    elif np.all(lnk == np.inf):
        return 1e99,0.0,[1e99],[0.0]
    elif np.all(np.logical_not(np.isfinite(lnk))):
        if len(t[lnk>-np.inf]) <= 0:
            tmin = t[0]
        else:
            tmin = t[lnk>-np.inf][0]
        return 1e99,t[lnk==-np.inf], [1e99], [tmin]

    n_t = t[np.abs(lnk) < np.inf]
    lnk = lnk[np.abs(lnk) < np.inf]
    if len(lnk) < 2:
        return lnk[0],np.zeros_like(n_t), lnk, n_t
    elif len(lnk) < 200:
        nk_intp = interp1d(n_t,lnk)
        n_t = np.linspace(n_t[0], n_t[-1], num=200)
        lnk = nk_intp(n_t)
    
    if tmax is None:
        tmax = np.max(n_t)
    filt = gaussian_filter1d(lnk, 1.0) 
    grad = np.gradient(filt, n_t)
    mu_norm = np.array(grad*n_t)
    #print('-------')
    #print(tmax)
    #print(np.where(n_t<=tmax))
    #print(n_t)
    if len(np.where(n_t<=tmax)) < 0:
        mu_max = []
        stable_islands = []
    else:
        try:
            mu_max = np.max(mu_norm[np.where(n_t<=tmax)])
        except ValueError:
            mu_max = np.max(mu_norm)
        stable_islands = n_t[np.where(mu_norm < 1)]
    return mu_max,stable_islands,mu_norm,n_t

def stable_cont(islands,dt):
    diff = np.round(np.diff(islands),6)
    dt = np.round(dt,6)
    idx = np.where(diff != dt)[0]
    u = np.insert(islands[idx],len(islands[idx]),islands[-1])
    l = np.insert(islands[idx+1],0,islands[0])
    return l+dt,u+dt

def process_dict(d,exclude=None,exclude_recurse=False):
    new_d = {}
    if exclude_recurse:
        r_ex = exclude
    else:
        r_ex = None
    for key,value in d.items():
        if isinstance(value,dict):
            new_d[key] = process_dict(value,r_ex)
        else:
            try:
                new_d[key] = np.array(value).tolist()
            except:
                if isinstance(value,np.generic):
                    new_d[key] = value.item()
                else:
                    new_d[key] = value
    if not exclude is None:
        for x in exclude:
            if x in new_d.keys():
                new_d.pop(x)
    return new_d

In [3]:
def analyse_run(params,data):

    # Convert amplitudes to occupation numbers
    k_values = get_kvals(params, None)
    times    = get_times(params, None)


    # 2D dimensionality array || dims = [N_k, N_t]
    # Each row contains a list of the occupation numbers over each timestep, for a given k-mode
    nk_sep = np.array([n_p(k_i, params, data, k_values, times, n=n_k) for k_i, k_v in enumerate(k_values)])
    # 1D array of the total occupation number over each timestep || dims = [N_t]
    # Each row contains a list of the occupation number over each timestep, for a given k-mode
    nk_tot = np.sum(nk_sep, axis=0)

    mu_max,stable_islands,mu_norm,ntimes = resonance_check(times,nk_tot)
    mu_norm = np.array(mu_norm)
    if len(mu_norm) > 2:
        dt = ntimes[1] - ntimes[0]
        if len(stable_islands) > 0:
            bins_l,bins_u = stable_cont(stable_islands,dt)
        else:
            bins_l = []
        # print(nk_tot)
        # plt.plot(times,nk_tot)
        # plt.yscale('log')
        # plt.show()
        if len(bins_l) > 1:
            spectrum_criteria = (bins_l[1:]/bins_u[:-1])/1.12
            if spectrum_criteria[0] > 1:
                r_spec = [bins_u[0]]
            else:
                try:
                    idx = np.where(spectrum_criteria>1)[0][0]
                except:
                    idx = -1
                r_spec = (bins_u[:idx])
        elif mu_max >= 1.0:
            r_spec = [ntimes[np.where(mu_norm>=1)[0][0]]]
    elif mu_max >= 1.0:
        r_spec = [ntimes[np.where(mu_norm>=1)[0][0]]]
    else:
       r_spec = [0]

    # plt.plot(ntimes,mu_norm)
    # plt.yscale("log")
    # plt.show()
    m_pi = params['m'][0][0]

    if np.any(mu_norm > 1):
        resonance = True
    else:
        resonance = False

    if resonance:
        tmin = ntimes[np.where(mu_norm>=1)[0][0]]
        res_list = []
        max_list = []
        nk_list = []
        for k,nk in zip(k_values,nk_sep):
            mu_max_k, stable_k, mu_norm_k, t_k = resonance_check(times, nk, tmax=1.12*tmin)
            if mu_max_k > 1:
                res_list.append(k)
            max_list.append(mu_max_k)
            nk_list.append(np.max(nk[times<=1.12*r_spec[0]]))
        if not np.all(mu_norm == 1e99):
            spec = np.array(max_list)*k_values*nk_list
            #plt.plot((k_values*m_pi*units.eV/constants.h).to('MHz').value,spec)

            if len(k_values[np.where(spec==np.max(spec))]) > 0:
                peak_k = k_values[np.where(spec==np.max(spec))][0] 
            else:
                peak_k = 0.0

            intp = interp1d(k_values,spec)
            k_int = np.logspace(np.log10(k_values[0]),np.log10(k_values[-1]),num=200)
            spec = intp(k_int)
            n_spec = [k_int,spec] #[np.where(np.isfinite(spec))]
            # plt.plot(k_values,max_list)
            # plt.show()
        else:
            peak_k = 0.0
            n_spec = [0.0,0.0]
            r_spec = [0.0]
    else:
        tmin = 0
        peak_k = 0
        mu_max = 0.0
        r_spec = [0.0]
        n_spec = [0.0,0.0]
    # print(f"Minimum size = {(tmin/m_pi/(1*units.eV)*constants.h*constants.c).to('km'):.2e}")
    # print(f"Peak frequency = {(peak_k*m_pi*units.eV/constants.h).to('MHz'):.2e}")
    # print(f"Bandwidth = {(fwhm*m_pi*units.eV/constants.h).to('MHz'):.2e}")

    out_dict = {}
    out_dict['F'] = params['F']
    out_dict['m_pi'] = m_pi
    out_dict['L4'] = params['L4']
    out_dict['p_t'] = params['p_t']
    out_dict['rmax'] = (tmin/m_pi/(1*units.eV)*constants.h*constants.c).to('km').value
    out_dict['peak_nu'] = (peak_k*m_pi*units.eV/constants.h).to('MHz').value
    out_dict['mu_max'] = mu_max
    out_dict['rmax_spectrum'] = (np.array(r_spec)/m_pi/(1*units.eV)*constants.h*constants.c).to('km').value
    out_dict['n_spectrum'] = [(n_spec[0]*m_pi*units.eV/constants.h).to('MHz').value,n_spec[1]]
    # if np.array(n_spec).size > 2:
    #     plt.plot(n_spec[0],n_spec[1])
    #     plt.yscale('log')
    #     plt.show()
    return out_dict


In [4]:
import cchardet
from json import JSONDecodeError
from pickle import UnpicklingError

def detect_bom(filename, verbosity=0):
    with open(filename, 'rb') as f:
            if verbosity >= 2:
                print('   BOM: ', f.read(4))
            raw = f.read(4)
            if raw.startswith(b'\xef\xbb\xbf'):
                return 'UTF-8'
            elif raw.startswith(b'\xff\xfe'):
                return 'UTF-16 LE'
            elif raw.startswith(b'\xfe\xff'):
                return 'UTF-16 BE'
    # Check for UTF-32
    try:
        with open(filename, encoding='utf-32') as f:
            content = f.read()
            return 'UTF-32'
    except UnicodeDecodeError:
        return None

def detect_encoding(filename, verbosity=0):
    file_bom = detect_bom(filename, verbosity)
    if file_bom is not None:
        return file_bom
    else:
        with open(filename, 'rb') as f:
            rawdata = f.read()
        result = cchardet.detect(rawdata)
        return result['encoding']

def print_encoding(file_base, ext_list=[''], verbosity=1):
    in_file_arr = [file_base+ext for ext in ext_list]
    for subfile in in_file_arr:
        if verbosity >= 1:
            print('   Filename: ', subfile)
        print('   Encoding: ', detect_encoding(subfile, verbosity))

In [ ]:
dict_list = []
if False:
    is_multiple_run_folder = True
    in_list_dir = output_dir
    in_list = [file_smooth, file_narrow, file_none, file_steep]
else:
    is_multiple_run_folder = False
    in_list_dir = output_dir
    in_list = list(set([os.path.splitext(in_file.name)[0].split('_funcs')[0].split('_plots')[0] for in_file in os.scandir(os.path.expanduser(in_list_dir)) if in_file.is_file()]))
    skip_list = []
    in_list = [filename for filename in in_list if not filename in skip_list]
#print('\n'.join(in_list[:5]))
print('---------------------------------')

skipped_list = []
missing_list = []
jsonerr_list = []
pickled_list = []
n_total = 0
n_saved = 0
for in_file in in_list:
    n_total += 1
    if is_multiple_run_folder:
        subfolder = '_'.join(in_file.split('_')[:-1])+'/'
    else:
        subfolder = ''
    in_folder = os.path.expanduser(os.path.join(output_dir,subfolder))
    
    #print(in_file)
    #print_encoding(os.path.join(in_folder, in_file))

    try:
        example_params, example_results, _, _ = load_single_result(in_folder, in_file)
    except FileNotFoundError as file_err:
        print('FileNotFoundError: %s' % in_file)
        #print(file_err)
        #raise file_err
        continue
    except JSONDecodeError as json_err:
        #print('JSONDecodeError: %s' % in_file)

        #full_file_path = os.path.join(in_folder, in_file)
        #ext_file_path = full_file_path + '.json'
        #encoding = print_encoding(ext_file_path)

        jsonerr_list.append(in_file)
        continue
    except UnpicklingError as unpickle_err:
        #print('UnpicklingError: %s' % in_file)

        #full_file_path = os.path.join(in_folder, in_file)
        #ext_file_path = full_file_path + '.npy'
        #encoding = print_encoding(ext_file_path)

        pickled_list.append(in_file)
        continue
    
    try:
        out = analyse_run(example_params, example_results)
        dict_list.append(process_dict(out))
        n_saved += 1
    except ValueError as val_err:
        print(val_err)
        skipped_list.append(in_file)
    except FileNotFoundError as file_err:
        missing_list.append(in_file)
    
    # Save progress every 1000 files loaded in
    if n_total % 1000 == 0:
        with open('astro_data.yaml','w') as out_file:
            yaml.dump(dict_list, out_file)

dumpfile = 'astro_data.yaml'
with open(dumpfile,'w') as out_file:
    yaml.dump(dict_list, out_file)

print('---------------------------------')
n_skipped = len(skipped_list)
n_missing = len(missing_list)
n_jsonerr = len(jsonerr_list)
n_pickled = len(pickled_list)
if n_total > 0:
    print('%d total runs loaded from %s' % (n_total, output_dir))
if n_saved > 0:
    print('%d runs saved to %s' % (n_saved, dumpfile))
if n_skipped > 0:
    print('ValueError files: %d \n' % n_skipped, skipped_list)
if n_missing > 0:
    print('FileNotFoundError files: %d \n' % n_missing, missing_list)
if n_jsonerr > 0:
    print('JSONDecodeError files: %d \n' % n_jsonerr, jsonerr_list)
if n_pickled > 0:
    print('UnpicklingError files: %d \n' % n_pickled, pickled_list)

---------------------------------


/tmp/ipykernel_1194411/3820884521.py:2: RuntimeWarning: invalid value encountered in log10
  lnk = np.log10(nk)
/tmp/ipykernel_1194411/3820884521.py:2: RuntimeWarning: divide by zero encountered in log10
  lnk = np.log10(nk)
/projects/illinois/eng/physics/nyunes/loane2/pyAxiverse/tools/../pyaxi_utils.py:1062: RuntimeWarning: overflow encountered in square
  n_k = lambda k, A, Adot, Im: ((k/2)*(np.abs(A)**2 + (np.abs(Adot)**2 / k**2)) - (1/2))
/projects/illinois/eng/physics/nyunes/loane2/pyAxiverse/tools/../pyaxi_utils.py:1062: RuntimeWarning: overflow encountered in multiply
  n_k = lambda k, A, Adot, Im: ((k/2)*(np.abs(A)**2 + (np.abs(Adot)**2 / k**2)) - (1/2))


In [ ]:
import glob
import shutil
import random

# Copy a random sampling of a dataset into a new folder, for testing purposes
def copy_sample_subset(output_dir, N=100, copy_path=None, is_multiple_run_folder=False):

    def get_new_file_name(in_file):

        if is_multiple_run_folder:
            subfolder = '_'.join(in_file.split('_')[:-1])+'/'
        else:
            subfolder = ''

        if copy_path is None:
            new_folder = os.path.expanduser(os.path.join(output_dir, subfolder, '..', os.path.basename(output_dir)+f'_sample_{N}'))
        else:
            new_folder = os.path.expanduser(copy_path)

        in_file_base = os.path.basename(in_file)
        #print('in_file_base: ', in_file_base)
        #print('new_folder: ', new_folder)
        new_file = os.path.join(os.path.abspath(new_folder), in_file_base)
        #print('           ', new_file)
        return new_file

    if is_multiple_run_folder:
        in_list_dir = output_dir
        in_list = [file_smooth, file_narrow, file_none, file_steep]
    else:
        in_list_dir = output_dir
        in_list = list(set([os.path.splitext(in_file.name)[0].split('_funcs')[0].split('_plots')[0] for in_file in os.scandir(os.path.expanduser(in_list_dir)) if in_file.is_file()]))
        skip_list = []
        in_list = [filename for filename in in_list if not filename in skip_list]
    
    #print(in_list[0])
    #print(get_new_file_name(in_list[0]))

    sub_list = random.sample(in_list, k=N)

    #print(sub_list)

    if True:
        for sub_list_file in sub_list:
            sub_file_set = glob.glob(os.path.expanduser(os.path.join(in_list_dir, sub_list_file+'*')))
            for sub_file_src in sub_file_set:
                #print('-------------------------')
                sub_file_dst = get_new_file_name(sub_file_src)
                #print('Copying : ', sub_file_src)
                #print(' ------>  ', sub_file_dst)
                sub_file_dst_dir = os.path.abspath(os.path.join(sub_file_dst, '..'))
                if not os.path.exists(sub_file_dst_dir):
                    os.makedirs(sub_file_dst_dir)
                shutil.copy(sub_file_src, sub_file_dst)
    
    print(f'Done copying files for N={N} runs from {output_dir}')



copy_sample_subset(output_dir, N=1000)
        

In [9]:
# Delete corrupted files
delete_corrupted = True
if delete_corrupted:
    corrupted_set = set(jsonerr_list).union(set(pickled_list))
    print('N: ', len(corrupted_set))
    print('corrupted_set: \n', corrupted_set)
    if True and len(corrupted_set) > 0:
        for file_stub in corrupted_set:
            # Specify the pattern for files to delete
            del_pattern = os.path.join(in_folder, file_stub+'*')

            # Find all matching files
            del_files = glob.glob(del_pattern)

            # Delete each file
            for file in del_files:
                os.remove(file)
        print(f'Deleted {len(corrupted_set)} files')

N:  100
corrupted_set: 
 {'piaxiverse_main1_SU3_71faff269891ec1c3b7f07302fc7fb0c3151da90', 'piaxiverse_main1_SU3_7258003a6607e89b04ca8298994651fc8ad6be34', 'piaxiverse_main1_SU3_714aa17ff2bafc668f059ec92a22f257af31a34a', 'piaxiverse_main1_SU3_72877f0e8a3e1ffee4ddc4d74c0bddde2bbc656f', 'piaxiverse_main1_SU3_72644afeb5051f0922c4ca95d3c1942bacae0100', 'piaxiverse_main1_SU3_724bac79545d10c1e9d07e7df3a2b481d86208ae', 'piaxiverse_main1_SU3_71dc00a550dd7d803a70d2771d33c98c6095e450', 'piaxiverse_main1_SU3_70f67403730e1cf89bfdbe0f5acac5b59cd2405d', 'piaxiverse_main1_SU3_71c5edba3f8bfaea1c442adc8c4a65b356cb081a', 'piaxiverse_main1_SU3_7085b83f7701ef16417364323f0cc172c3a5f1c9', 'piaxiverse_main1_SU3_71d4267421ca07a927153e867517aa9df7a49e27', 'piaxiverse_main1_SU3_71729849d6e1ef16d9a82db6d828d5b2a1c10123', 'piaxiverse_main1_SU3_71ae04506168d725a71adbfea15631da47bf4049', 'piaxiverse_main1_SU3_723f8c8639129446ec81104c7784c29129501afa', 'piaxiverse_main1_SU3_722bad514842426280c44cc46e413e0bcac6d488',